# Planner

When we know exactly which semantic function we want to invoke in our app, we can just specify it in our code. As we grow our library of interesting and useful semantic functions, we might not want a specific button in our UI to invoke each function. Wouldn't it be cool if the AI could determine which semantic function best fits the user's question?

In this tutorial, we'll look at three planners: Action Planner, Sequential Planner, and Stepwise Planner.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.20.230821.4-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;
// the following is an abbreviated version of the code in Tutorial0 that reads from the settings file and sets up the kernel
MySettings settings = Settings.LoadFromFile();
IKernel kernel = Settings.SetupSemanticKernel(settings);

# Action Planner

The Action Planner chooses one - and only one - semantic function from the list of functions you provide to the kernel.

Before you run the following code, open the CelebratePlugin.cs file in the Plugins folder. Here, you'll see more than one SKFunction are defined (e.g. GenerateOccasionCard and GenerateGiftIdeas). When creating the plan, we'll first use the user's question to determine the best function to call. We'll then execute the function chosen by the AI. 

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;

#!import Plugins/CelebratePlugin.cs

// load the kernel up with multiple functions to choose from
var celebratePlugin = kernel.ImportSkill(new CelebratePlugin(kernel));

// create a planner and ask it to create a plan for us based on the user's prompt
ActionPlanner planner = new(kernel);
Plan plan = await planner.CreatePlanAsync("I have no idea what to get my dad for father's day!");

Console.WriteLine(plan.ToJson(true));

// with the plan in hand, we can now execute it and get the result
SKContext result = await plan.InvokeAsync();

Console.WriteLine(result);

I know the response is long, but I want you to see the plan generated.

The response you are looking for is all the way at the bottom where it generated a list of ideas for father's day gifts even though we didn't direct the kernel to invoke the GenerateGiftIdeas function. There are two AI calls happening here:
1. We pass the prompt and a list of our available plugins/function to an instance of ActionPlanner. The ActionPlanner takes care of passing descriptions of each of our funtions to AI and determine which best fits the user's provided input.
2. The second call passes our Father's Day prompt to the GenerateGiftIdeas function - and no other function.

# Sequential Planner

When you know you only want to invoke the single best function, the Action Planner is great! What if we aren't sure of that and we want the AI to select all the necessary functions to answer a user's question. That's where the Sequential Planner shines! After providing it with a number of functions to choose from and a more complex scenario, it will choose the best functions in the correct sequence.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

// register plugins for getting thematic travel destinations, hotel availability, rental car availability, points of interest, and flight availability
string travelDestinations = "give me a comma-separated list of travel destinations given these criteria: {{$input}}";
kernel.CreateSemanticFunction(travelDestinations, description: "");

string touristActivities = "give me a list of popular tourist activities for these destinations: {{$input}}";
kernel.CreateSemanticFunction(touristActivities, description: "");

string drinkRecipes = "give me a a drink recipe given these criteria: {{$input}}";
kernel.CreateSemanticFunction(drinkRecipes);

string drinkRecipes = "give me a a drink recipe given these criteria: {{$input}}";
kernel.CreateSemanticFunction(drinkRecipes);

// user's prompt: book a trip
string destination = "I want to book a two-week trip to somewhere warm and sunny and get some ideas for things to do while I'm there.";

// create an instance of a planner and provide a connection to the kernel
SequentialPlanner planner = new(kernel);

Plan plan = await planner.CreatePlanAsync(destination);

Console.WriteLine(plan.ToJson(true));

# Exercise

